In [34]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

## Define dataset and variables

In [35]:
case = "fanv3_debugging"
path = "/glade/scratch/jinmuluo/archive/fanv3clm2000/lnd/hist/" 
start_date = date(2000, 1, 1)
end_date = date(2000, 12, 1)
delta = end_date - start_date

In [36]:
fieldsh2 = ["N2O_DENITRIFY_TOTAL", "N2O_NITRIFY_TOTAL", "NOx_DENITRIFY_TOTAL", "NOx_NITRIFY_TOTAL", 
            "N2_TOTAL", "area"]
fieldsh1 = ["NITRITE_G1", "NITRITE_G2", "NITRITE_G3", "NITRITE_S0", "NITRITE_S1", "NITRITE_S2",
            "NITRITE_S3", "NITRITE_F1", "NITRITE_F2",  "NITRITE_F3",  "NITRITE_F4",'area']
fieldsh0 = ["GPP", "FERT_NO3_TO_SOIL", "MANURE_NO3_TO_SOIL", "MANURE_NITRITE_RUNOFF", 'FERT_NITRITE_RUNOFF', 
            "FERT_NITRITE_TO_SOIL", "MANURE_NITRITE_TO_SOIL", 'area']

## (1) Nitrification flux from fanv2

In [37]:
fanCLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h0" + "." + month +".nc")

In [38]:
def preprocess(ds, fields = fieldsh0):
    return(ds[fields])

def fix_time(ds):  
    date0 = ds['time'][0].values
    date1 = ds['time'][-1].values
    # ds['time'] =xr.cftime_range(str(yr0),periods=ndays,freq='D')
    ds['time'] = pd.date_range(str(date0),str(date1),freq='MS') 
    
    return ds

dsCLM = fix_time(xr.open_mfdataset(fanCLM, decode_times=True, preprocess=preprocess))
# dsCLM

In [39]:
ds = (dsCLM["FERT_NO3_TO_SOIL"] + dsCLM["MANURE_NO3_TO_SOIL"]) * dsCLM['area']*1e6
ds = ds.sum(dim=['lat', 'lon'])
fanv2_flux = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    fanv2_flux = fanv2_flux + ds[i].values * day * 24 * 3600
fanv2_flux = fanv2_flux * 1e-12

print(fanv2_flux, "Tg/year")

43.7976837072 Tg/year


## (2) NOX and N2O emission 

In [41]:
fanCLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h2" + "." + month +".nc")

In [42]:
def preprocess(ds, fields = fieldsh2):
    return(ds[fields])

def fix_time(ds):  
    date0 = ds['time'][0].values
    date1 = ds['time'][-1].values
    # ds['time'] =xr.cftime_range(str(yr0),periods=ndays,freq='D')
    ds['time'] = pd.date_range(str(date0),str(date1),freq='MS') 
    
    return ds

dsCLM = fix_time(xr.open_mfdataset(fanCLM, decode_times=True, preprocess=preprocess))
# dsCLM

In [43]:
ds_n2o = (dsCLM["N2O_DENITRIFY_TOTAL"] + dsCLM["N2O_NITRIFY_TOTAL"]) * dsCLM['area']*1e6
ds_nox = (dsCLM["NOx_DENITRIFY_TOTAL"] + dsCLM["NOx_NITRIFY_TOTAL"]) * dsCLM['area']*1e6
ds_n2 = dsCLM["N2_TOTAL"] * dsCLM['area']*1e6

ds_n2o = ds_n2o.sum(dim=['lat', 'lon'])
ds_nox = ds_nox.sum(dim=['lat', 'lon'])
ds_n2 = ds_n2.sum(dim=['lat', 'lon'])

n2o_total = 0.0
nox_total = 0.0
n2_total = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    n2o_total = n2o_total + ds_n2o[i].values * day * 24 * 3600
    nox_total = nox_total + ds_nox[i].values * day * 24 * 3600
    n2_total = n2_total + ds_n2[i].values * day * 24 * 3600
    
n2o_total = n2o_total * 1e-12
nox_total = nox_total * 1e-12
n2_total = n2_total * 1e-12

print('N2O:', n2o_total, "Tg/year")
print('NOx:', nox_total, "Tg/year")
print('N2', n2_total, "Tg/year")

N2O: 0.6380716615875 Tg/year
NOx: 0.1506265680515625 Tg/year
N2 0.062234409311718746 Tg/year


## (3) Diffusion and Runoff

In [44]:
fanCLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h0" + "." + month +".nc")

In [45]:
def preprocess(ds, fields = fieldsh0):
    return(ds[fields])

def fix_time(ds):  
    date0 = ds['time'][0].values
    date1 = ds['time'][-1].values
    # ds['time'] =xr.cftime_range(str(yr0),periods=ndays,freq='D')
    ds['time'] = pd.date_range(str(date0),str(date1),freq='MS') 
    
    return ds

dsCLM = fix_time(xr.open_mfdataset(fanCLM, decode_times=True, preprocess=preprocess))
# dsCLM

In [46]:
ds_soil = (dsCLM["FERT_NITRITE_TO_SOIL"] + dsCLM["MANURE_NITRITE_TO_SOIL"]) * dsCLM['area']*1e6
ds_runoff = (dsCLM["MANURE_NITRITE_RUNOFF"] + dsCLM["FERT_NITRITE_RUNOFF"]) * dsCLM['area']*1e6
ds_soil = ds_soil.sum(dim=['lat', 'lon'])
ds_runoff = ds_runoff.sum(dim=['lat', 'lon'])

soil = 0.0
runoff = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    soil = soil + ds_soil[i].values * day * 24 * 3600
    runoff = runoff + ds_runoff[i].values * day * 24 * 3600
    
soil = soil * 1e-12
runoff = runoff * 1e-12

print("diffusion to soil", soil, "Tg/year")
print("surface runoff", runoff, "Tg/year")

diffusion to soil -340.9931586852 Tg/year
surface runoff 196.0786574568 Tg/year


## (4) Nitrite pools

In [47]:
fanCLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h1" + "." + month +".nc")

In [48]:
dsCLM = xr.open_dataset(fanCLM[-1])

In [49]:
ds_nitrite = (dsCLM["NITRITE_G1"] + dsCLM["NITRITE_G2"] + dsCLM["NITRITE_G3"] + dsCLM["NITRITE_S0"] + dsCLM["NITRITE_S1"] + 
              dsCLM["NITRITE_S2"] + dsCLM["NITRITE_S3"] + dsCLM["NITRITE_F1"] + dsCLM["NITRITE_F2"] + dsCLM["NITRITE_F3"] +
              dsCLM["NITRITE_F4"]) * dsCLM["area"] * 1e6

nitrite = ds_nitrite.sum(dim=['lat', 'lon']).values
# g to Tg
nitrite = nitrite * 1e-12
print("nitrite pools: ", nitrite, "Tg")

nitrite pools:  [184.2984] Tg


## N Balance

In [50]:
d = fanv2_flux - n2o_total - nox_total - n2_total - runoff - soil - nitrite
print(d)

[3.562851]
